# Spectrum

In [1]:
import pandas as pd
import astropy.units as u
import numpy as np
import re
import matplotlib.pyplot as plt
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
from pandas.plotting import scatter_matrix

In [2]:
catalog = pd.read_csv("markeddata/gevtevsimbad_TeVspectrum.txt", sep='\t', index_col=0)
catalog.shape

(68, 199)

In [3]:
xmm_catalog = pd.read_csv("data/gevtevsimbadxmm.txt", sep='\t', index_col=0)
xmm_catalog.shape

(67, 237)

In [4]:
# columns contain spectrums in xmm
xmm_spectrum_columns = [
#     "SC_RA",
#     "SC_DEC",
#     "SC_POSERR",
    "SC_EP_1_FLUX",
    "SC_EP_2_FLUX",
    "SC_EP_3_FLUX",
    "SC_EP_4_FLUX",
    "SC_EP_5_FLUX",
#     "SC_EP_1_FLUX_ERR",
#     "SC_EP_2_FLUX_ERR",
#     "SC_EP_3_FLUX_ERR",
#     "SC_EP_4_FLUX_ERR",
#     "SC_EP_5_FLUX_ERR",
    ]
xmm_spectrum_columns = list(map(lambda x: "xmm_" + x, xmm_spectrum_columns))

In [5]:
catalog = catalog.join(xmm_catalog.loc[catalog.index, xmm_spectrum_columns])
catalog

,level_0,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,tev_30TeV,xmm_SC_EP_1_FLUX,xmm_SC_EP_2_FLUX,xmm_SC_EP_3_FLUX,xmm_SC_EP_4_FLUX,xmm_SC_EP_5_FLUX
0,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,3.341668e-11,2.440195e-12,1.828992e-13,5.610705e-15,6.128219e-17,4.258610e-17,5.177130e-16,6.523510e-16,0.000000e+00,1.173750e-15
1,92,b' ',b' ',b' ',b' ',b'HESS J1841-055 ',b' ',b' ',b' ',b' ',...,6.310590e-11,1.207092e-11,8.365091e-13,4.356098e-14,2.865129e-15,3.273380e-16,0.000000e+00,2.955490e-16,2.293610e-15,2.526860e-14
2,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,3.341668e-11,2.440195e-12,1.828992e-13,5.610705e-15,6.128219e-17,4.258610e-17,5.177130e-16,6.523510e-16,0.000000e+00,1.173750e-15
3,86,b' ',b'1FGL J1746.4-2849c',b'1FHL J1746.3-2851 ',b'2FGL J1746.6-2851c',b'PWN G0.13-0.11 ',b' ',b' ',b'3EG J1746-2851',b' ',...,1.647138e-11,8.320851e-14,2.778103e-14,1.856180e-15,3.184437e-16,1.757200e-17,1.882010e-17,1.424720e-16,7.676820e-15,9.757430e-14
4,83,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,3.341668e-11,2.440195e-12,1.828992e-13,5.610705e-15,6.128219e-17,4.258610e-17,5.177130e-16,6.523510e-16,0.000000e+00,1.173750e-15
5,85,b' ',b'1FGL J1746.4-2849c',b'1FHL J1746.3-2851 ',b'2FGL J1746.6-2851c',b'PWN G0.13-0.11 ',b' ',b' ',b'3EG J1746-2851',b' ',...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.757200e-17,1.882010e-17,1.424720e-16,7.676820e-15,9.757430e-14
6,86,b' ',b'1FGL J1746.4-2849c',b'1FHL J1746.3-2851 ',b'2FGL J1746.6-2851c',b'PWN G0.13-0.11 ',b' ',b' ',b'3EG J1746-2851',b' ',...,1.647138e-11,8.320851e-14,2.778103e-14,1.856180e-15,3.184437e-16,NaN,NaN,NaN,NaN,NaN
7,103,b'0FGL J2032.2+4122',b'1FGL J2032.2+4127 ',b'1FHL J2032.1+4125 ',b'2FGL J2032.2+4126 ',b'LAT PSR J2032+4127 ',b' ',b'1AGL J2032+4102',b' ',b' ',...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.173680e-17,1.546940e-16,1.521890e-15,6.733740e-15,3.592550e-15
8,103,b'0FGL J2032.2+4122',b'1FGL J2032.2+4127 ',b'1FHL J2032.1+4125 ',b'2FGL J2032.2+4126 ',b'LAT PSR J2032+4127 ',b' ',b'1AGL J2032+4102',b' ',b' ',...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.173680e-17,1.546940e-16,1.521890e-15,6.733740e-15,3.592550e-15
9,11,b'0FGL J0240.3+6113',b'1FGL J0240.5+6113 ',b'1FHL J0240.2+6113 ',b'2FGL J0240.5+6113 ',b'LS I+61 303 ',b' ',b'1AGL J0242+6111',b' ',b'EGR J0240+6112 ',...,3.033065e-11,5.807977e-13,5.926255e-14,1.818965e-15,2.823374e-16,0.000000e+00,0.000000e+00,2.872300e-15,0.000000e+00,3.058640e-14


Columns contained spectrum in gev:

In [6]:
gev_spectrum_columns = [
    'gev_nuFnu10000_100000',
    'gev_nuFnu1000_3000',
    'gev_nuFnu100_300',
    'gev_nuFnu3000_10000',
    'gev_nuFnu300_1000',
    'gev_nuFnu30_100',
]

Columns contained spectrum in simbad:

In [7]:
s_spectrum_columns = [
    's_FLUX_U', 
    's_FLUX_B', 
    's_FLUX_V', 
    's_FLUX_R', 
    's_FLUX_I',
    's_FLUX_G', 
    's_FLUX_J', 
    's_FLUX_H',
    's_FLUX_K', 
    's_FLUX_u',
    's_FLUX_g', 
    's_FLUX_r',
    's_FLUX_i',    
    's_FLUX_z',
    ]

Columns contained spectrum in tev:

In [8]:
tev_spectrum_columns =  [
    'tev_0.3TeV', 
    'tev_1TeV', 
    'tev_3TeV',  
    'tev_10TeV',  
    'tev_30TeV' 
]

In [9]:
s_other_columns = ['s_MAIN_ID',
                 's_class', 
                 's_nan_spectra'  
                 ]

In [10]:
gev_other_columns = ['gev_1FGL_Name',
                     'gev_CLASS1',
                    ]

In [11]:
tev_other_columns = ['tev_fermi_names', 
                     'tev_classes',
                    ]

In [12]:
spectrum_columns = gev_spectrum_columns + s_spectrum_columns + tev_spectrum_columns
spectrum_with_xmm_columns = spectrum_columns + xmm_spectrum_columns
other_columns = s_other_columns + gev_other_columns + tev_other_columns + ['marked']

In [13]:
spectrums = catalog[spectrum_columns + other_columns].copy()

The next cell computes simbad spectrum using this formula:
$$ F_{simbad} = 10^{-flux_{simbad}}$$

In [14]:
spectrums[s_spectrum_columns] = 10 ** (-spectrums[s_spectrum_columns])

The next cell computes TeV spectrum using:
$$F_{TeV} = 1.6*E^{2}*Flux$$

In [15]:
for i in tev_spectrum_columns:
    if len(re.findall("\d+", i)) == 2:
        E = 0.3
    else:
        E = int(re.findall("\d+", i)[0])
    spectrums[i] = 1.6 * E ** 2 * spectrums[i]

In [16]:
spectrums = spectrums.fillna(0)

In [17]:
display(spectrums)

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,s_FLUX_U,s_FLUX_B,s_FLUX_V,s_FLUX_R,...,tev_10TeV,tev_30TeV,s_MAIN_ID,s_class,s_nan_spectra,gev_1FGL_Name,gev_CLASS1,tev_fermi_names,tev_classes,marked
0,2.938044e-12,7.446349e-11,4.520060e-12,3.103834e-11,7.171538e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,8.977128e-13,8.824635e-14,b'SSTGC 519103',Be,False,b' ',b' ',b'',b'unid',0
1,2.098420e-11,2.096862e-11,1.266728e-12,2.274123e-11,7.566604e-12,0.0,0.000000e+00,6.918312e-12,3.090294e-11,0.000000e+00,...,6.969757e-12,4.125786e-12,b'LS IV -05 11',B,False,b' ',b'PWN ',b'',b'unid',0
2,2.938044e-12,7.446349e-11,4.520060e-12,3.103834e-11,7.171538e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,8.977128e-13,8.824635e-14,b'GCIRS 1W',B,False,b' ',b' ',b'',b'unid',0
3,4.706814e-12,5.332763e-11,5.145968e-13,2.923599e-11,3.387356e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.969888e-13,4.585590e-13,b'[MCD2010] 18',O,False,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
4,2.938044e-12,7.446349e-11,4.520060e-12,3.103834e-11,7.171538e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,8.977128e-13,8.824635e-14,b'GCIRS 8',O,False,b' ',b' ',b'',b'unid',0
5,4.706814e-12,5.332763e-11,5.145968e-13,2.923599e-11,3.387356e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,b'[MCD2010] 13',O,False,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
6,4.706814e-12,5.332763e-11,5.145968e-13,2.923599e-11,3.387356e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.969888e-13,4.585590e-13,b'[MCD2010] 13',O,False,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
7,2.845686e-12,4.600289e-11,1.418847e-11,3.224835e-11,3.170300e-11,0.0,7.585778e-14,8.317639e-14,1.122019e-12,0.000000e+00,...,0.000000e+00,0.000000e+00,b'[MT91] 213',B0Ve,False,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',1
8,2.845686e-12,4.600289e-11,1.418847e-11,3.224835e-11,3.170300e-11,0.0,4.073800e-13,9.772367e-13,5.011870e-12,8.912505e-12,...,0.000000e+00,0.000000e+00,b'Schulte 21',B0Ve,False,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',0
9,5.929003e-12,8.840736e-11,1.353836e-10,4.055951e-11,1.317918e-10,0.0,5.370312e-12,2.454711e-12,1.778279e-11,6.456549e-11,...,2.910345e-13,4.065659e-13,b'LS I +61 303',B0Ve,False,b'1FGL J0240.5+6113 ',b'HMB ',b'',b'bin',1


Normalization:

In [18]:
spectrums[spectrum_columns] = spectrums[spectrum_columns].div(spectrums.sum(axis=1), axis=0)

The cell displays all information we have about spectrums now.

In [19]:
display(spectrums)

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,s_FLUX_U,s_FLUX_B,s_FLUX_V,s_FLUX_R,...,tev_10TeV,tev_30TeV,s_MAIN_ID,s_class,s_nan_spectra,gev_1FGL_Name,gev_CLASS1,tev_fermi_names,tev_classes,marked
0,1.353665e-02,3.430807e-01,2.082558e-02,1.430050e-01,3.304191e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,4.136093e-03,4.065834e-04,b'SSTGC 519103',Be,False,b' ',b' ',b'',b'unid',0
1,1.333478e-02,1.332488e-02,8.049650e-04,1.445132e-02,4.808333e-03,0.0,0.000000e+00,4.396364e-03,1.963782e-02,0.000000e+00,...,4.429056e-03,2.621804e-03,b'LS IV -05 11',B,False,b' ',b'PWN ',b'',b'unid',0
2,1.681789e-03,4.262423e-02,2.587363e-03,1.776690e-02,4.105117e-02,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,5.138669e-04,5.051379e-05,b'GCIRS 1W',B,False,b' ',b' ',b'',b'unid',0
3,1.139862e-02,1.291450e-01,1.246213e-03,7.080159e-02,8.203253e-02,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,7.192259e-04,1.110505e-03,b'[MCD2010] 18',O,False,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
4,1.280898e-02,3.246382e-01,1.970608e-02,1.353177e-01,3.126572e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.913755e-03,3.847272e-04,b'GCIRS 8',O,False,b' ',b' ',b'',b'unid',0
5,5.245202e-03,5.942750e-02,5.734588e-04,3.258014e-02,3.774818e-02,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,b'[MCD2010] 13',O,False,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
6,5.223892e-03,5.918606e-02,5.711290e-04,3.244777e-02,3.759482e-02,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.296152e-04,5.089350e-04,b'[MCD2010] 13',O,False,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
7,2.845686e-12,4.600289e-11,1.418847e-11,3.224835e-11,3.170300e-11,0.0,7.585778e-14,8.317639e-14,1.122019e-12,0.000000e+00,...,0.000000e+00,0.000000e+00,b'[MT91] 213',B0Ve,False,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',1
8,7.519406e-04,1.215575e-02,3.749144e-03,8.521264e-03,8.377162e-03,0.0,1.076456e-04,2.582238e-04,1.324330e-03,2.355030e-03,...,0.000000e+00,0.000000e+00,b'Schulte 21',B0Ve,False,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',0
9,5.929002e-12,8.840736e-11,1.353836e-10,4.055951e-11,1.317918e-10,0.0,5.370312e-12,2.454711e-12,1.778279e-11,6.456548e-11,...,2.910344e-13,4.065659e-13,b'LS I +61 303',B0Ve,False,b'1FGL J0240.5+6113 ',b'HMB ',b'',b'bin',1


In [20]:
# spectrums.to_csv("markeddata/normalized_spectra.txt", index=False)

In [21]:
def calculate_spectra(catalog_name, xmm_catalog_name, output_catalog_name, 
                      s_spectrum_columns, gev_spectrum_columns, tev_spectrum_columns, xmm_spectrum_columns,
                      spectrum_columns, s_other_columns, gev_other_columns, tev_other_columns):
    other_columns = s_other_columns + gev_other_columns + tev_other_columns + ['marked']
    catalog = pd.read_csv("markeddata/" + catalog_name + ".txt", sep='\t', index_col=0)
    xmm_catalog = pd.read_csv("data/{}.txt".format(xmm_catalog_name), sep='\t', index_col=0)
    catalog = catalog.join(xmm_catalog.loc[catalog.index, xmm_spectrum_columns])
    spectrum_columns = gev_spectrum_columns + s_spectrum_columns + tev_spectrum_columns + xmm_spectrum_columns
    spectrums = catalog[spectrum_columns + other_columns].copy()
    if len(s_spectrum_columns) > 0:
        spectrums[s_spectrum_columns] = 10 ** (-spectrums[s_spectrum_columns])
    for i in tev_spectrum_columns:
        if len(re.findall("\d+", i)) == 2:
            E = 0.3
        else:
            E = int(re.findall("\d+", i)[0])
        spectrums[i] = 1.6 * E ** 2 * spectrums[i]
    spectrums = spectrums.fillna(0)
    spectrums[spectrum_columns] = spectrums[spectrum_columns].div(spectrums.sum(axis=1), axis=0)
    spectrums = spectrums.fillna(0)
    display(spectrums)
    spectrums.to_csv("markeddata/" + output_catalog_name + "_normalized.txt", index=False)

* adding xmm spectrum columns to the gevtevsimbad, tevsimbad and gevsimbad spectrums

In [22]:
calculate_spectra(catalog_name="gevtevsimbad_TeVspectrum", 
                  xmm_catalog_name="gevtevsimbadxmm",
                  output_catalog_name="gevtevsimbadxmm_spectrum",
                  s_spectrum_columns=[], 
                  gev_spectrum_columns=gev_spectrum_columns, 
                  tev_spectrum_columns=tev_spectrum_columns,
                  xmm_spectrum_columns=xmm_spectrum_columns,
                  spectrum_columns=spectrum_columns,
                  s_other_columns=[],
                  gev_other_columns=gev_other_columns,
                  tev_other_columns=tev_other_columns)
calculate_spectra(catalog_name="gevtevsimbad_TeVspectrum", 
                  xmm_catalog_name="gevtevsimbadxmm",
                  output_catalog_name="gevtevsimbadxmm_spectrum",
                  s_spectrum_columns=[], 
                  gev_spectrum_columns=gev_spectrum_columns, 
                  tev_spectrum_columns=tev_spectrum_columns,
                  xmm_spectrum_columns=xmm_spectrum_columns,
                  spectrum_columns=spectrum_columns,
                  s_other_columns=[],
                  gev_other_columns=gev_other_columns,
                  tev_other_columns=tev_other_columns)
calculate_spectra(catalog_name="tevsimbad_TeVspectrum",
                  xmm_catalog_name="tevsimbadxmm",
                  output_catalog_name="tevsimbadxmm_spectrum",
                  s_spectrum_columns =s_spectrum_columns, 
                  gev_spectrum_columns=[], 
                  tev_spectrum_columns=tev_spectrum_columns, 
                  xmm_spectrum_columns=xmm_spectrum_columns,
                  spectrum_columns=spectrum_columns,
                  s_other_columns=s_other_columns,
                  gev_other_columns=[],
                  tev_other_columns=tev_other_columns)
calculate_spectra(catalog_name="gevsimbad", 
                  xmm_catalog_name="gevsimbadxmm",
                  output_catalog_name="gevsimbadxmm_spectrum",
                  s_spectrum_columns=s_spectrum_columns, 
                  gev_spectrum_columns=gev_spectrum_columns, 
                  tev_spectrum_columns=[], 
                  xmm_spectrum_columns=xmm_spectrum_columns,
                  spectrum_columns=spectrum_columns,
                  s_other_columns=s_other_columns,
                  gev_other_columns=gev_other_columns,
                  tev_other_columns=[])

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,...,xmm_SC_EP_1_FLUX,xmm_SC_EP_2_FLUX,xmm_SC_EP_3_FLUX,xmm_SC_EP_4_FLUX,xmm_SC_EP_5_FLUX,gev_1FGL_Name,gev_CLASS1,tev_fermi_names,tev_classes,marked
0,1.491289e-02,3.779609e-01,2.294287e-02,1.575440e-01,3.640121e-01,0.0,2.442470e-02,1.981747e-02,1.336835e-02,4.556601e-03,...,2.161580e-07,2.627802e-06,3.311196e-06,0.000000e+00,5.957707e-06,b' ',b' ',b'',b'unid',0
1,1.677427e-01,1.676181e-01,1.012592e-02,1.817880e-01,6.048563e-02,0.0,7.264130e-02,1.543872e-01,9.629071e-02,5.571457e-02,...,2.616662e-06,0.000000e+00,2.362548e-06,1.833457e-05,2.019912e-04,b' ',b'PWN ',b'',b'unid',0
2,1.491289e-02,3.779609e-01,2.294287e-02,1.575440e-01,3.640121e-01,0.0,2.442470e-02,1.981747e-02,1.336835e-02,4.556601e-03,...,2.161580e-07,2.627802e-06,3.311196e-06,0.000000e+00,5.957707e-06,b' ',b' ',b'',b'unid',0
3,3.752703e-02,4.251767e-01,4.102837e-03,2.330961e-01,2.700711e-01,0.0,1.891079e-02,1.061463e-03,3.189540e-03,2.367867e-03,...,1.401001e-07,1.500511e-07,1.135917e-06,6.120663e-05,7.779516e-04,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
4,1.491289e-02,3.779609e-01,2.294287e-02,1.575440e-01,3.640121e-01,0.0,2.442470e-02,1.981747e-02,1.336835e-02,4.556601e-03,...,2.161580e-07,2.627802e-06,3.311196e-06,0.000000e+00,5.957707e-06,b' ',b' ',b'',b'unid',0
5,3.865521e-02,4.379589e-01,4.226181e-03,2.401037e-01,2.781902e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.443119e-07,1.545621e-07,1.170066e-06,6.304670e-05,8.013393e-04,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
6,3.755860e-02,4.255344e-01,4.106289e-03,2.332922e-01,2.702983e-01,0.0,1.892670e-02,1.062356e-03,3.192223e-03,2.369859e-03,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
7,2.845686e-12,4.600289e-11,1.418847e-11,3.224835e-11,3.170300e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,5.173680e-17,1.546940e-16,1.521890e-15,6.733740e-15,3.592550e-15,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',1
8,2.240690e-02,3.622262e-01,1.117199e-01,2.539231e-01,2.496290e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,4.073749e-07,1.218059e-06,1.198334e-05,5.302139e-05,2.828770e-05,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',0
9,5.929003e-12,8.840736e-11,1.353836e-10,4.055951e-11,1.317918e-10,0.0,4.367614e-12,9.292763e-13,8.533807e-13,2.910345e-13,...,0.000000e+00,0.000000e+00,2.872300e-15,0.000000e+00,3.058640e-14,b'1FGL J0240.5+6113 ',b'HMB ',b'',b'bin',1


,s_FLUX_U,s_FLUX_B,s_FLUX_V,s_FLUX_R,s_FLUX_I,s_FLUX_G,s_FLUX_J,s_FLUX_H,s_FLUX_K,s_FLUX_u,...,xmm_SC_EP_2_FLUX,xmm_SC_EP_3_FLUX,xmm_SC_EP_4_FLUX,xmm_SC_EP_5_FLUX,s_MAIN_ID,s_class,s_nan_spectra,tev_fermi_names,tev_classes,marked
0,0.000000e+00,0.000000e+00,0.000000e+00,6.031191e-05,0.000000e+00,1.433189e-04,1.699820e-03,3.238938e-03,7.769672e-03,0.0,...,6.595268e-04,1.377706e-04,1.729550e-04,0.000000e+00,b'UCAC4 105-013663',Be,False,b'',b'other',0
1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.376178e-04,1.956257e-02,8.698124e-01,0.0,...,7.753807e-06,1.695367e-05,1.731059e-06,1.197008e-04,b'MSX6C G331.5487-00.5364',Be,False,b'',b'unid',0
2,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.605737e-06,2.453511e-03,6.162949e-01,0.0,...,2.044118e-06,3.997852e-06,2.017363e-04,0.000000e+00,b'SSTGC 519103',Be,False,b'',b'unid',0
3,0.000000e+00,2.770110e-02,4.597241e-02,0.000000e+00,0.000000e+00,0.000000e+00,1.325859e-01,1.708036e-01,2.053510e-01,0.0,...,2.691242e-05,4.489883e-05,9.232349e-05,5.306250e-05,b'RMC 141',B,False,b'',b'pwn',0
4,0.000000e+00,5.303197e-04,6.173589e-04,3.424035e-04,4.737377e-04,6.167912e-04,3.841832e-04,2.597397e-04,2.440832e-04,0.0,...,6.658365e-04,1.390887e-04,1.746097e-04,0.000000e+00,b'[AAA97b] J053648.7-691700',B,False,b'',b'other',0
5,0.000000e+00,1.826715e-05,2.196194e-05,2.247352e-05,4.233227e-05,3.255909e-05,1.554788e-05,1.628065e-05,1.451010e-05,0.0,...,6.443576e-05,1.075002e-04,2.210479e-04,1.270463e-04,b'OGLE LMC-ECL-20994',B,False,b'',b'pwn',0
6,9.557094e-03,1.906892e-03,1.623029e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,...,6.594099e-04,1.377462e-04,1.729244e-04,0.000000e+00,b'[ST92] 2-62B',B,False,b'',b'other',0
7,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.517758e-03,3.310722e-02,8.568855e-02,0.0,...,6.342122e-06,5.952411e-05,9.544715e-05,9.505656e-05,b'[KKM2013] 7',B,False,b'',"b'pwn,snr'",0
8,4.381570e-03,1.869088e-03,2.640155e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,...,6.463373e-04,1.350154e-04,1.694962e-04,0.000000e+00,b'[ST92] 2-62',B,False,b'',b'other',0
9,2.967648e-03,9.581029e-04,3.796756e-03,9.894915e-03,4.522857e-01,4.113507e-03,8.154784e-02,1.575479e-01,2.502724e-01,0.0,...,2.359762e-06,3.936864e-06,8.095200e-06,4.652679e-06,b'HTR 13',B,False,b'',b'pwn',0


,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,s_FLUX_U,s_FLUX_B,s_FLUX_V,s_FLUX_R,...,xmm_SC_EP_2_FLUX,xmm_SC_EP_3_FLUX,xmm_SC_EP_4_FLUX,xmm_SC_EP_5_FLUX,s_MAIN_ID,s_class,s_nan_spectra,gev_1FGL_Name,gev_CLASS1,marked
0,1.257344e-12,2.414663e-11,8.859199e-12,1.480012e-11,2.684172e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,5.337160e-17,3.305730e-16,3.044530e-15,3.128880e-15,b'[KRL2007b] 194',Be,True,b'1FGL J1636.4-4737c',b'snr ',0
1,4.367979e-06,8.042724e-05,1.744995e-05,3.972280e-05,7.974043e-05,0.0,8.700521e-05,3.626980e-05,8.119804e-05,1.022224e-04,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'HD 328990',Be,False,b'1FGL J1651.5-4602c',b' ',0
2,1.434933e-02,8.725673e-02,9.646709e-02,5.886473e-02,8.262877e-02,0.0,4.191539e-05,2.250987e-05,1.788025e-04,8.172849e-04,...,9.280713e-05,4.235945e-05,0.000000e+00,1.790372e-06,b'UCAC2 5051621',Be,False,b' ',b'spp ',0
3,5.414068e-02,1.236920e-01,4.429135e-01,1.212983e-01,1.887740e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,9.619466e-07,3.488502e-07,3.462881e-06,9.904365e-05,b'SSTGC 519103',Be,False,b'1FGL J1745.6-2900c',b'spp ',0
4,4.641164e-02,2.321283e-01,2.709139e-01,1.452088e-01,3.042503e-01,0.0,0.000000e+00,2.830525e-05,2.623415e-05,0.000000e+00,...,3.333689e-06,3.898443e-05,1.229669e-04,1.901747e-04,b'LIN 341',Be,False,b'1FGL J0101.3-7257 ',b'GAL ',0
5,1.523725e-03,6.399917e-03,1.623092e-02,4.016086e-03,8.842059e-03,0.0,1.508756e-01,3.456357e-02,5.231411e-02,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'LS VI -04 15',B,False,b' ',b' ',0
6,1.573251e-04,2.620224e-03,8.954646e-03,1.090295e-03,3.881067e-03,0.0,0.000000e+00,6.987534e-03,1.528706e-02,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'LS I +56 58',B,False,b' ',b' ',0
7,1.435295e-05,1.313128e-01,4.130346e-01,7.424117e-02,3.705391e-01,0.0,0.000000e+00,2.135673e-04,4.462050e-04,3.158892e-04,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'PG 1101+529',B,False,b' ',b' ',0
8,1.144422e-05,2.401992e-01,3.359969e-01,8.313828e-02,3.247328e-01,0.0,0.000000e+00,0.000000e+00,1.592146e-02,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'GALEX J175340.5-500741',B,False,b'1FGL J1754.0-5002 ',b'bcu ',0
9,1.940893e-02,1.180237e-01,1.304816e-01,7.962058e-02,1.117639e-01,0.0,2.011610e-03,2.058463e-03,7.647928e-03,1.457282e-02,...,1.255312e-04,5.729549e-05,0.000000e+00,2.421661e-06,b'HD 306182',B,False,b' ',b'spp ',0


* contain objects, which have spectrum information from catalogs (gev, tev, simbad) and are in their intersection, but were not found in xmm catalog

In [23]:
gevtevsimbadxmm = pd.read_csv("markeddata/gevtevsimbadxmm_spectrum_normalized.txt",)
gevtevsimbadxmm

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,...,xmm_SC_EP_1_FLUX,xmm_SC_EP_2_FLUX,xmm_SC_EP_3_FLUX,xmm_SC_EP_4_FLUX,xmm_SC_EP_5_FLUX,gev_1FGL_Name,gev_CLASS1,tev_fermi_names,tev_classes,marked
0,1.491289e-02,3.779609e-01,2.294287e-02,1.575440e-01,3.640121e-01,0.0,2.442470e-02,1.981747e-02,1.336835e-02,4.556601e-03,...,2.161580e-07,2.627802e-06,3.311196e-06,0.000000e+00,5.957707e-06,b' ',b' ',b'',b'unid',0
1,1.677427e-01,1.676181e-01,1.012592e-02,1.817880e-01,6.048563e-02,0.0,7.264130e-02,1.543872e-01,9.629071e-02,5.571457e-02,...,2.616662e-06,0.000000e+00,2.362548e-06,1.833457e-05,2.019912e-04,b' ',b'PWN ',b'',b'unid',0
2,1.491289e-02,3.779609e-01,2.294287e-02,1.575440e-01,3.640121e-01,0.0,2.442470e-02,1.981747e-02,1.336835e-02,4.556601e-03,...,2.161580e-07,2.627802e-06,3.311196e-06,0.000000e+00,5.957707e-06,b' ',b' ',b'',b'unid',0
3,3.752703e-02,4.251767e-01,4.102837e-03,2.330961e-01,2.700711e-01,0.0,1.891079e-02,1.061463e-03,3.189540e-03,2.367867e-03,...,1.401001e-07,1.500511e-07,1.135917e-06,6.120663e-05,7.779516e-04,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
4,1.491289e-02,3.779609e-01,2.294287e-02,1.575440e-01,3.640121e-01,0.0,2.442470e-02,1.981747e-02,1.336835e-02,4.556601e-03,...,2.161580e-07,2.627802e-06,3.311196e-06,0.000000e+00,5.957707e-06,b' ',b' ',b'',b'unid',0
5,3.865521e-02,4.379589e-01,4.226181e-03,2.401037e-01,2.781902e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.443119e-07,1.545621e-07,1.170066e-06,6.304670e-05,8.013393e-04,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
6,3.755860e-02,4.255344e-01,4.106289e-03,2.332922e-01,2.702983e-01,0.0,1.892670e-02,1.062356e-03,3.192223e-03,2.369859e-03,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,b'1FGL J1746.4-2849c',b'pwn ',b'',b'unid',0
7,2.845686e-12,4.600289e-11,1.418847e-11,3.224835e-11,3.170300e-11,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,5.173680e-17,1.546940e-16,1.521890e-15,6.733740e-15,3.592550e-15,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',1
8,2.240690e-02,3.622262e-01,1.117199e-01,2.539231e-01,2.496290e-01,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,4.073749e-07,1.218059e-06,1.198334e-05,5.302139e-05,2.828770e-05,b'1FGL J2032.2+4127 ',b'PSR ',b'',b'unid',0
9,5.929003e-12,8.840736e-11,1.353836e-10,4.055951e-11,1.317918e-10,0.0,4.367614e-12,9.292763e-13,8.533807e-13,2.910345e-13,...,0.000000e+00,0.000000e+00,2.872300e-15,0.000000e+00,3.058640e-14,b'1FGL J0240.5+6113 ',b'HMB ',b'',b'bin',1


In [29]:
# calculate_spectra(catalog_name="gevtev_TeVspectrum", 
#                   s_spectrum_columns=[], 
#                   gev_spectrum_columns=gev_spectrum_columns, 
#                   tev_spectrum_columns=tev_spectrum_columns, 
#                   s_other_columns=[],
#                   gev_other_columns=gev_other_columns,
#                   tev_other_columns=tev_other_columns)
# calculate_spectra(catalog_name="tevsimbad_TeVspectrum", 
#                   s_spectrum_columns =s_spectrum_columns, 
#                   gev_spectrum_columns=[], 
#                   tev_spectrum_columns=tev_spectrum_columns, 
#                   s_other_columns=s_other_columns,
#                   gev_other_columns=[],
#                   tev_other_columns=tev_other_columns)
# calculate_spectra(catalog_name="gevsimbad", 
#                   s_spectrum_columns=s_spectrum_columns, 
#                   gev_spectrum_columns=gev_spectrum_columns, 
#                   tev_spectrum_columns=[], 
#                   s_other_columns=s_other_columns,
#                   gev_other_columns=gev_other_columns,
#                   tev_other_columns=[])